# Discretizing Thermal Diffusion
In this example, we use the Loewner framework to create a finite-dimensional reduced-order model directly from a partial differential equation. This example uses the basics discussed in the [mass-spring-damper example](../MassSpringDamper/demo.ipynb). Therefore, it is recommended to finish that example before continuoing here.

We consider the one-dimensional heat equation with radiation of the form

$$
\tag{5.1}
\frac{\partial T}{\partial t} = \kappa \frac{\partial^{2} T}{\partial x^{2}} - \beta T,
$$

with $\kappa = \frac{\lambda}{\rho \gamma}$, where $\lambda$ is the thermal conductivity, $\rho$ is the density and $\gamma$ is the specific heat capacity of the material, and $\beta T$ models the heat loss due to radiation. The solution $T(t, x)$ models the heat in the one-dimensional domain. For the boundary of the domain $x \in [x_{\min}, x_{\max}]$, we have external forcing on the left side and temperature observations on the right

$$
T(t, x_{\min}) = u(t) \quad \text{and} \quad y(t) = T(t, x_{\max}).
$$

This linear partial differential equation $(5.1)$ with the boundary condition and observation can be equivalently be described in the frequency domain via

$$ \tag{5.2}
Y(s) = G(s) U(s) = \underbrace{\left( e^{-a \sqrt{(s + \mu)\kappa^{-1}}} \right)}_{G(s)} U(s),
$$

where $a = x_{\max} - x_{\min}$ is the length of the domain, $Y(s)$ and $U(s)$ are the Laplace transforms of the heat inflow and the observagtions, and $G(s)$ is the systems transfer function.

The goal in this example is to use evaluations the transfer function in $(5.2)$ to derive a reduced-order linear time-invariant finite-dimensional system of ordinary differential equations that describe the dynamics of the partial differential equation $(5.1)$ without constructing an intermediate discretization.
The advantage of working with the transfer function is that the discretization that we create is directly targeted towards the input-to-output (heat inflow to observation) behavior of the dynamical system $(5.1)$ without the intermediate step of describing the complete internal system behavior. The finite-dimensional reduced-order model will be given via the transfer function

$$ \tag{5.3}
\widehat{G}(s) = \widehat{\mathbf{c}}^{\mathsf{T}} (s \widehat{\mathbf{E}} - \widehat{\mathbf{A}})^{-1} \widehat{\mathbf{b}},
$$

which we can interprete in the time domain as a system of generalized ordinary differential equations of the form

$$
\begin{aligned}
  \widehat{\mathbf{E}} \dot{\widehat{\mathbf{q}}}(t) & = \widehat{\mathbf{A}}\widehat{\mathbf{q}}(t) + \widehat{\mathbf{b}} u(t), \\
  \widehat{y}(t) & = \widehat{\mathbf{c}}^{\mathsf{T}} \widehat{\mathbf{q}}(t),
\end{aligned}
$$

with $\widehat{\mathbf{A}}, \widehat{\mathbf{E}} \in \mathbb{R}^{r \times r}$, $\widehat{\mathbf{b}}, \widehat{\mathbf{c}} \in \mathbb{R}^{r}$ and the state-space dimension $r$ small.

First, we import some standard Python packages for our computations and the visualization.

In [ ]:
import numpy as np
import numpy.linalg as la
import scipy.sparse as sparse
import scipy.linalg as spla
import matplotlib.pyplot as plt

System construction and some visualization routines are implemented in the file [utils.py](./utils.py).

In [ ]:
import utils

utils.configure_matplotlib(latex_is_installed = True)

Besides the `SISO` class that implements linear systems with `A, b, c, E` matrices and transfer function $(5.3)$, this file also implements the class `ThermalDiffusion` that creates a model of the thermal diffusion process with the transfer function $(5.2)$.

Let's create an instance of the thermal diffusion model. The parameters have been chosen for the case of a cast iron rod:

In [ ]:
thermal_model = utils.ThermalDiffusion(
    length       = 0.5,
    conductivity = 55.0,
    density      = 6.55,
    capacity     = 460.548,
    radiation    = 1.0
)

Note that `fom_model` implements an infinite-dimensional model for which at this point we do not have a finite-dimensional realization available yet.
We can only evaluate its transfer function, which we will use in the following to construct the finite-dimensional reduced-order representation.

# Step 1: Generate sampling data
First, we need to sample the scalar transfer function in complex points. While any point in $\mathbb{C}$ that is not a pole of $(5.2)$ would be suitable, the go to choice is the positive imaginary axis. We choose the evaluation points logarithmically equidistant to cover a reasonably large range of frequencies. Feel free to modify the number of sampling points `num_samples` in the code below, but note that we will double the amount of generated data later for the generation of a reduced-order model with real matrices.

In [ ]:
# Number of sampling points (must be an even number).
num_samples = 100

# Sample the transfer function.
training_frequencies = np.logspace(-2, 2, num_samples)
training_responses   = thermal_model.transfer_function(1j * training_frequencies)

To get a first impression of the data and the transfer function we need to fit, we visualize the generated data by plotting the magnitudes of the frequency responses over the real-valued frequencies.

In [ ]:
ax = utils.plot_response(
    training_frequencies,
    training_responses,
    linestyle = '',
    marker    = '.',
    label     = "Transfer function data"
)
ax.legend(loc = "lower left")
plt.show()

# Step 2: Data preparations
As mentioned in the beginning, we will use the Loewner framework to generate a suitable linear reduced-order model. Before we prepare the data for the construction of the Loewner matrices, we need to have a closer look at the transfer function $G(s)$ in $(5.2)$. We can see that

$$
\overline{G(s)} = G(\overline{s})
$$

holds. To be save, let's verify this for our generated data points.

In [ ]:
# Sample in the complex conjugate points.
conjugate_response = thermal_model.transfer_function(-1j * training_frequencies)

# Verify complex conjugation of data.
np.allclose(training_responses, conjugate_response.conj())

This means that the transfer function has the same property as the transfer function of a first-order model with real matrices. Therefore, we should generate a real finite-dimensional model by including the complex conjugates of our training data.

Now that we know the complete training data to include, we can separate the complete data set into left and right $(\mu_{\operatorname{\ell}}, g_{\operatorname{\ell}})$ and $(\mu_{\operatorname{r}}, g_{\operatorname{r}})$, where additionally we want to include the complex conjugate data so that 

$$
\begin{aligned}
  \mu_{\operatorname{\ell}} & = \{ \mu_{\operatorname{\ell}, 1}, \overline{\mu_{\operatorname{\ell}, 1}}, \mu_{\operatorname{\ell}, 2}, \overline{\mu_{\operatorname{\ell}, 2}}, \ldots  \}, &
  g_{\operatorname{\ell}} & = \{ g_{\operatorname{\ell}, 1}, \overline{g_{\operatorname{\ell}, 1}}, g_{\operatorname{\ell}, 2}, \overline{g_{\operatorname{\ell}, 2}}, \ldots \}, \\
  \mu_{\operatorname{r}} & = \{ \mu_{\operatorname{r}, 1}, \overline{u_{\operatorname{r}, 1}}, u_{\operatorname{r}, 2}, \overline{u_{\operatorname{r}, 2}}, \ldots  \}, &
  g_{\operatorname{r}} & = \{ g_{\operatorname{r}, 1}, \overline{g_{\operatorname{r}, 1}}, g_{\operatorname{r}, 2}, \overline{g_{\operatorname{r}, 2}}, \ldots \}.
\end{aligned}
$$

holds. Remember that left and right data should be of equal size.

In [ ]:
# Function to arrange the complex conjugate structure correctly.
def insert_conjugates(arr):
    """From an array [x1, x2, ...], form [x1, conj(x1), x2, conj(x2), ...]"""
    new_arr       = np.empty(len(arr) * 2, dtype = complex)
    new_arr[::2]  = arr
    new_arr[1::2] = arr.conj()
    return new_arr

# Data splitting.
mu_l = insert_conjugates(1j * training_frequencies[::2])
mu_r = insert_conjugates(1j * training_frequencies[1::2])
g_l  = insert_conjugates(training_responses[::2])
g_r  = insert_conjugates(training_responses[1::2])

print(
    "Size of left data",
    (len(mu_l), len(g_l)),
    "Size of right data",
    (len(mu_r), len(g_r)),
    sep="\n"
)

**Imporant:** Remember that in the Loewner framework, we will create matrices with dimensions of the size of these data sets. This does not correspond to the final order of the model that we are constructing but it is an intermediate step. You can experiment with the sizes of the data sets and the created Loewner matrices by changing the `num_samples` parameter above. To avoid dimensional problems later on, always choose `num_samples` to be an even number.

# Step 3: Set up the Loewner matrices
Next, we create our Loewner matrices of the form

$$
\begin{aligned}
    \mathbb{L}
    &= \begin{bmatrix}
        \frac{g_{\operatorname{\ell}, 1} - g_{\operatorname{r}, 1}}{\mu_{\operatorname{\ell}, 1} - \mu_{\operatorname{r}, 1}} & \frac{g_{\operatorname{\ell}, 1} - g_{\operatorname{r}, 2}}{\mu_{\operatorname{\ell}, 1} - \mu_{\operatorname{r}, 2}} & \cdots
        \\[.25cm]
        \frac{g_{\operatorname{\ell}, 2} - g_{\operatorname{r}, 1}}{\mu_{\operatorname{\ell}, 2} - \mu_{\operatorname{r}, 1}} & \frac{g_{\operatorname{\ell}, 2} - g_{\operatorname{r}, 2}}{\mu_{\operatorname{\ell}, 2} - \mu_{\operatorname{r}, 2}} &  \cdots
        \\[.25cm]
        \vdots & \vdots & \ddots
    \end{bmatrix} \in \mathbb{C}^{n_{\operatorname{s}} \times n_{\operatorname{s}}},
    &
    \mathbb{L}_{\operatorname{s}} &= \begin{bmatrix}
        \frac{\mu_{\operatorname{\ell}, 1}g_{\operatorname{\ell}, 1} - \mu_{\operatorname{r}, 1}g_{\operatorname{r}, 1}}{\mu_{\operatorname{\ell}, 1} - \mu_{\operatorname{r}, 1}} & \frac{\mu_{\operatorname{\ell}, 1}g_{\operatorname{\ell}, 1} - \mu_{\operatorname{r}, 2}g_{\operatorname{r}, 2}}{\mu_{\operatorname{\ell}, 1} - \mu_{\operatorname{r}, 2}} & \cdots
        \\[.25cm]
        \frac{\mu_{\operatorname{\ell}, 2}g_{\operatorname{\ell}, 2} - \mu_{\operatorname{r}, 1}g_{\operatorname{r}, 1}}{\mu_{\operatorname{\ell}, 2} - \mu_{\operatorname{r}, 1}} & \frac{\mu_{\operatorname{\ell}, 2} g_{\operatorname{\ell}, 2} - \mu_{\operatorname{r}, 2} g_{\operatorname{r}, 2}}{\mu_{\operatorname{\ell}, 2} - \mu_{\operatorname{r}, 2}} &  \cdots
        \\[.25cm]
        \vdots & \vdots & \ddots
    \end{bmatrix} \in \mathbb{C}^{n_{\operatorname{s}} \times n_{\operatorname{s}}},
    \\
    \mathbf{B}_{\mathbb{L}}
    &= \begin{bmatrix}
        g_{\operatorname{\ell}, 1} \\
        g_{\operatorname{\ell}, 2} \\
        \vdots
    \end{bmatrix} \in \mathbb{C}^{n_{\operatorname{s}}},
    &
    \mathbf{C}_{\mathbb{L}}
    & = \left[\begin{array}{ccc}
        g_{\operatorname{r}, 1} & g_{\operatorname{r}, 2} & \cdots
    \end{array}\right] \in \mathbb{C}^{n_{\operatorname{s}}},
\end{aligned}
$$

by using our prepared and separated data sets. Here, the number $n_{\operatorname{s}}$ is given by the parameter `num_samples` from above (or equivalently the size of the left and right data sets). Note that at this point, the constructed Loewner matrices are still complex.

In [ ]:
# Change vector format for broadcasting.
k    = len(mu_l)
g_l  = g_l.reshape((k, 1))
mu_l = mu_l.reshape((k, 1))

# Construct matrices.
L  = (g_l - g_r) / (mu_l - mu_r)
Ls = ((mu_l * g_l) - (mu_r * g_r)) / (mu_l - mu_r)
BL = g_l.reshape((k,))
CL = g_r

# Verify size of the Loewner system.
for label, arr in zip(["L", "Ls", "BL", "CL"], [L, Ls, BL, CL]):
    print(
        f"Matrix '{label}' dimensions:{arr.shape}"
    )

Before moving on to construct the reduced-order approximation, we need take care of the realification of the system matrices. Therefore, we to apply a state-space transformation to the Loewner matrices using

$$
    \mathbf{J}
    = \mathbf{I}_{\frac{n_{\operatorname{s}}}{2}} \otimes \frac{1}{\sqrt{2}}
    \left[\begin{array}{rr}
        1 & \mathrm{j} \\ 1 & -\mathrm{j}
    \end{array}\right],
$$

which results in an equivalent system with real matrices.

In [ ]:
# Transformation matrix.
J = sparse.kron(
    sparse.eye(num_samples // 2), \
    sparse.coo_matrix((1 / np.sqrt(2)) * np.array([[1, 1j], [1, -1j]]))
)

# State-space transformation.
L_real  = (J.conj().T @ (L @ J)).real
Ls_real = (J.conj().T @ (Ls @ J)).real
BL_real = (J.conj().T @ BL).real
CL_real = (CL @ J).real

# Step 4: Rank truncation and reduced-order model construction
To create our low-dimensional model, we need to truncate the Loewner matrices appropriately using two singular value decompositions.


In [ ]:
# Compute the singular value decompositions of the pencil matrices.
Phi_1, s1, Psi_1T = la.svd(np.vstack((Ls_real, L_real)))  # [Ls; L]
Phi_2, s2, Psi_2T = la.svd(np.hstack((Ls_real, L_real)))  # [Ls, L]

# Visualization singular value decay to determine rank.
utils.plot_singular_values(s1, s2)
plt.show()

We see that we can reach machine precision accuracy to represent the given data with the model already at order about $20$. The truncation to smaller orders will provide reasonable models but result in a larger least-squares error with respect to the given data. Looking back to the computed frequency response, we see that the transfer function magnitude for higher frequencies becomes very small. Since we can expect the Loewner approximation to be accurate up to the relative order of magnitude chosen by our tolerance, we need to choose a reasonably small tolerance to represent all the data well. 

**Important:** You cannot choose the tolerance too small, otherwise the matrix pencil $\lambda \widehat{\mathbf{E}} - \widehat{\mathbf{A}}$ becomes singular. Warnings about ill-conditioning in the evaluation of the transfer function are indicators.

In [ ]:
# Determine the truncation rank.
tol = 9.0e-12
r   = np.count_nonzero(s1 > s1[1] * tol)

print("Size of reduced-order model:", r)

With all done, we can finally construct our reduced-order finite-dimensional approximation of the thermal diffusion model.

In [ ]:
# Construct reduced-order matrices.
Ar = -Phi_2[:, 0:r].T @ (Ls_real @ Psi_1T[:, 0:r])
Er = -Phi_2[:, 0:r].T @ (L_real @ Psi_1T[:, 0:r])
br = Phi_2[:, 0:r].T @ BL_real
cr = CL_real @ Psi_1T[:, 0:r]

# Define the ROM.
loewner_model = utils.SISO(A = Ar, b = br, c = cr, E = Er)

# Summary and evaluations

## Frequency evaluation
With our finite-dimensional model computed, we should have a closer look at what we really obtained. First, let's see how well we are approximating our original object of interest, the transfer function $G(s)$. To do so, we compute the frequency response of both the original transfer function $G(s)$ from $(5.2)$ and the transfer function of our finite-dimensional approximation over the frequency range $[10^{-2}, 10^{2}]$ rad/s. We measure the error in a pointwise relative sense

$$
\operatorname{relerr}(\omega) = \frac{\lvert G(\mathfrak{j} \omega) - \widehat{G}(\mathfrak{j} \omega) \rvert}{\lvert G(\mathfrak{j} \omega) \rvert}.
$$

In [ ]:
# Number of test sampling points.
num_test = 500

# Compute frequency responses.
test_frequencies   = np.logspace(-2, 2, num_test)
test_responses_fom = thermal_model.transfer_function(1j * test_frequencies)
test_responses_rom = loewner_model.transfer_function(1j * test_frequencies)

# Compute relative error.
rom_relative_error = np.abs(test_responses_fom - test_responses_rom) / np.abs(
    test_responses_fom
)

# Visualize frequency responses.
axes = utils.plot_comparison(
    test_frequencies, test_responses_fom, test_responses_rom, rom_relative_error
)
axes[0].legend(loc="lower left")
plt.show()

For small enough truncation tolerance, there is barely any visible difference between the original transfer function and our finite-dimensional approximation. Only the relative error reveals that for higher frequencies, the approximation quality decreases but this has to be expected due to the fast decay in the magnitude of the frequency responses.

## Time domain simulation
Typically, we are less interested in the frequency domain formulation of a thermal diffusion process rather than its time domain description, which we can use for simulations of the system. Since we directly computed the matrices representing the transfer function via the Loewner framework

$$
\widehat{G}(s) = \widehat{\mathbf{c}}^{\mathsf{T}} (s \widehat{\mathbf{E}} - \widehat{\mathbf{A}})^{-1} \widehat{\mathbf{b}},
$$

the corresponding time domain description is given by

$$
\begin{aligned}
  \widehat{\mathbf{E}} \dot{\widehat{\mathbf{q}}}(t) & = \widehat{\mathbf{A}}\widehat{\mathbf{q}}(t) + \widehat{\mathbf{b}} u(t), \\
  \widehat{y}(t) & = \widehat{\mathbf{c}}^{\mathsf{T}} \widehat{\mathbf{q}}(t).
\end{aligned}
$$

We can verify if our reduced-order model will give us accurate time domain simulations without testing against the original dynamical system by checking too conditions:
* The reduced-order model is asymptotically stable.
* The transfer function of the reduced-order model has a small approximation error in the $\mathcal{H}_{\infty}$-norm (supremum / worst case error norm).

If these two conditions hold, it follows from the equivalence between the time and frequency domain, for the potential errors in simulations that

$$
\lVert y - \widehat{y} \rVert_{L_{2}} \leq \lVert G - \widehat{G} \rVert_{\mathcal{H}_{\infty}} \cdot \lVert u \rVert_{L_{2}},
$$

which bounds the error between the true time domain output signal and its approximation by the error we make in the approximation of the transfer function in the $\mathcal{H}_{\infty}$-norm and the energy that is given to the system via the input $u(t)$.

First, we check the asymptotic stability by computing the eigenvalues of the reduced-order matrix pencil $\lambda \widehat{\mathbf{E}} - \widehat{\mathbf{A}}$. If all eigenvalues lie in the open left half-plane, the model is asymptotically stable. It may happen that your reduced-order model is not stable. In this case, try to change the order of the model via the relative tolerance above and recompute.

In [ ]:
# Compute the eigenvalues.
eigs = spla.eigvals(Ar, Er)

# Visualize the eigenvalues in the complex plane.
utils.plot_eigenvalues(eigs)
plt.show()

For asymptotically stable models with real coefficients, the $\mathcal{H}_{\infty}$-norm is defined to as

$$
\lVert G \rVert_{\mathcal{H}_{\infty}} = \sup_{\omega > 0} \lVert G(\mathfrak{j} \omega) \rVert_{2}.
$$

We can use the frequency responses computed earlier and take the maximum error over these, which will be a highly accurate approximation to the $\mathcal{H}_{\infty}$-norm error.

In [ ]:
hinf_err = np.max(np.abs(test_responses_fom - test_responses_rom))

print("Approximate H-infinity error: ", hinf_err)

With the model being asymptotically stable and a small $\mathcal{H}_{\infty}$-norm error, we know that for admissible input signals $u(t)$, the simulations computed with the reduced-order model will be highly accurate approximations of the true behavior of the thermal diffusion process $(5.1)$. As final task in this example, we compute a time simulation using the reduced-order model for a sinusoidal input signal.

In [ ]:
# Compute time simulation.
test_simulation = loewner_model.time_simulation(
    t_span = [0, 10],
    q0     = np.zeros((r,)),
    input  = lambda t: np.sin(t)
)

utils.plot_simulation(test_simulation)
plt.show()

Physically the simulation results make a lot of sense since we have applied a sinusoidal input signal and the offset in the beginning results from the fact that the heat inflow happens at the opposite end of the point where we measure the temperature.